In [19]:
import dash 
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd


In [20]:
#City Data
data_files = {
    "Value Index": r"C:\Users\Jatab\bootcamp\Backups\df_value_index_avg.csv",
    "Rental Index": r"C:\Users\Jatab\bootcamp\Backups\df_rental_index_avg.csv"
}

def load_dataset(path):
    df = pd.read_csv(path)
    df = df.drop(columns=['_id','RegionID','SizeRank', 'RegionType'], errors='ignore')
    df = df.rename(columns={'RegionName': 'City'})  # 👈 The key line
    return df
dataframes = {name: load_dataset(path)for name, path in data_files.items()}

print(dataframes)

{'Value Index':                 City StateName           2000           2001           2002  \
0      United States       NaN  123244.341354  132216.059078  141407.454785   
1       New York, NY        NY  225898.640331  253963.007913  286441.831977   
2    Los Angeles, CA        CA  234232.790582  259460.074000  293963.604298   
3        Chicago, IL        IL  154897.139217  168219.844533  181690.066329   
4         Dallas, TX        TX  127330.335326  129510.347174  132673.619987   
..               ...       ...            ...            ...            ...   
890       Zapata, TX        TX            NaN            NaN            NaN   
891    Ketchikan, AK        AK            NaN            NaN            NaN   
892        Craig, CO        CO   98565.042098  106098.631320  112146.270120   
893       Vernon, TX        TX            NaN            NaN            NaN   
894       Lamesa, TX        TX            NaN            NaN            NaN   

              2003           2004  

In [21]:
#Dash App
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("City Trend Viewer"),

    dcc.Dropdown(
        id='data-type',
        options=[{"label": k, "value": k} for k in dataframes.keys()],
        value="Rental Index",
        placeholder="Select data type"
    ),

    dcc.Dropdown(
        id='city-selector',
        placeholder="Select a city"
    ),

    dcc.Graph(id='line-graph')
])

In [22]:
# Populate cities based on selected dataset
@app.callback(
    Output('city-selector', 'options'),
    Input('data-type', 'value')
)
def update_city_dropdown(data_type):
    df = dataframes[data_type]
    cities = df['City'].unique()
    return [{"label": city, "value": city} for city in cities]


In [23]:
# Update graph when city is selected
@app.callback(
    Output('line-graph', 'figure'),
    [Input('data-type', 'value'),
     Input('city-selector', 'value')]
)
def update_graph(data_type, selected_city):
    df = dataframes[data_type]
    if selected_city is None:
        return {}

    city_df = df[df['City'] == selected_city].drop(columns=['City'])
    city_df = city_df.T.reset_index()
    city_df.columns = ['Date', 'Value']
    city_df = city_df[city_df['Date'] != 'City'] 

    fig = px.line(city_df, x='Date', y='Value', title=f'{data_type} for {selected_city}')
    fig.update_layout(xaxis_title="Date", yaxis_title=data_type)
    return fig
if __name__ == '__main__':
    app.run(debug=True, port=8051)